# Restaurant Review Classifiers

## Overview
This notebook demonstrates a multi-output classification model for restaurant reviews, predicting food, service, and atmosphere ratings.

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [3]:
class RestaurantReviewClassifier:
    def __init__(self, model='logistic_regression'):
        self.tfidf_vectorizer = TfidfVectorizer(
            stop_words='english',
            max_features=5000,
            ngram_range=(1, 2)
        )
        self.label_encoders = {
            'food': LabelEncoder(),
            'service': LabelEncoder(),
            'atmosphere': LabelEncoder()
        }

        # Initialize classifier based on input model
        if model == 'logistic_regression':
            base_classifier = LogisticRegression(max_iter=1000)
        elif model == 'random_forest':
            base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
        elif model == 'svm':
            base_classifier = SVC(probability=True, kernel='linear')
        else:
            raise ValueError("Model not supported. Choose from 'logistic_regression', 'random_forest', 'svm'.")

        self.classifier = MultiOutputClassifier(base_classifier)

    def preprocess_data(self, df):
        # Drop rows with NaN in text column
        df = df.dropna(subset=['text'])

        # Fill NaN in categorical columns with 'None'
        columns = ['food', 'service', 'atmosphere']
        for col in columns:
            if col not in df.columns:
                df[col] = 'None'
            df.loc[:, col] = df[col].fillna('None')

        return df

    def train(self, df):
        # Preprocess data
        df = self.preprocess_data(df)

        # Vectorize text
        X = self.tfidf_vectorizer.fit_transform(df['text'].astype(str))

        # Encode labels dynamically
        y_dict = {}
        for col in ['food', 'service', 'atmosphere']:
            unique_labels = df[col].unique()
            self.label_encoders[col].fit(unique_labels)
            y_dict[col] = self.label_encoders[col].transform(df[col])

        y = pd.DataFrame(y_dict)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Train classifier
        self.classifier.fit(X_train, y_train)

        # Predict
        y_pred = self.classifier.predict(X_test)

        # Evaluate
        print("\nModel Performance Metrics:")
        for i, col in enumerate(['food', 'service', 'atmosphere']):
            print(f"\n{col.capitalize()} Classification:")
            classes = self.label_encoders[col].classes_

            accuracy = accuracy_score(y_test.iloc[:, i], y_pred[:, i])
            print(f"Accuracy: {accuracy:.2%}")

            print(classification_report(
                y_test.iloc[:, i],
                y_pred[:, i],
                labels=range(len(classes)),
                target_names=classes
            ))

    def predict(self, texts):
        # Convert texts to strings and handle potential NaN
        texts = [str(text) if pd.notna(text) else '' for text in texts]

        # Vectorize input
        X = self.tfidf_vectorizer.transform(texts)

        # Predict
        predictions = self.classifier.predict(X)

        # Decode predictions
        results = []
        for pred in predictions:
            result = {
                col: self.label_encoders[col].inverse_transform([p])[0]
                for col, p in zip(['food', 'service', 'atmosphere'], pred)
            }
            results.append(result)

        return results


In [4]:
# Logistic Regression
classifier_lr = RestaurantReviewClassifier(model='logistic_regression')

# Random Forest
classifier_rf = RestaurantReviewClassifier(model='random_forest')

# Support Vector Machine
classifier_svm = RestaurantReviewClassifier(model='svm')


In [11]:
df = pd.read_csv('../raw_data/clean_data.csv')  # Load your dataset

df_low_score = df[df['rating'] < 5]

# Sample 500 reviews with a score of 5
df_score_5_sample = df[df['rating'] == 5].sample(n=300, random_state=42)

# Combine the two DataFrames
df_filtered = pd.concat([df_low_score, df_score_5_sample])

# Reset the index if needed
df_filtered.reset_index(drop=True, inplace=True)

df = df_filtered

# Train using Logistic Regression
classifier_lr.train(df)

# Train using Random Forest
classifier_rf.train(df)

# Train using SVM
classifier_svm.train(df)



Model Performance Metrics:

Food Classification:
Accuracy: 68.10%
              precision    recall  f1-score   support

    Negative       0.69      0.33      0.45        66
        None       1.00      0.04      0.07        27
    Positive       0.68      0.97      0.80       139

    accuracy                           0.68       232
   macro avg       0.79      0.45      0.44       232
weighted avg       0.72      0.68      0.61       232


Service Classification:
Accuracy: 71.98%
              precision    recall  f1-score   support

    Negative       0.79      0.55      0.65        62
        None       0.66      0.77      0.71        87
    Positive       0.75      0.80      0.77        83

    accuracy                           0.72       232
   macro avg       0.73      0.70      0.71       232
weighted avg       0.73      0.72      0.72       232


Atmosphere Classification:
Accuracy: 77.59%
              precision    recall  f1-score   support

    Negative       0.00      

/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo


Model Performance Metrics:

Food Classification:
Accuracy: 67.67%
              precision    recall  f1-score   support

    Negative       0.64      0.24      0.35        66
        None       0.71      0.19      0.29        27
    Positive       0.68      0.98      0.80       139

    accuracy                           0.68       232
   macro avg       0.68      0.47      0.48       232
weighted avg       0.67      0.68      0.61       232


Service Classification:
Accuracy: 70.26%
              precision    recall  f1-score   support

    Negative       0.69      0.39      0.49        62
        None       0.78      0.79      0.78        87
    Positive       0.65      0.84      0.73        83

    accuracy                           0.70       232
   macro avg       0.70      0.67      0.67       232
weighted avg       0.71      0.70      0.69       232


Atmosphere Classification:
Accuracy: 78.88%
              precision    recall  f1-score   support

    Negative       0.00      

/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo


Model Performance Metrics:

Food Classification:
Accuracy: 71.12%
              precision    recall  f1-score   support

    Negative       0.67      0.47      0.55        66
        None       0.75      0.11      0.19        27
    Positive       0.72      0.94      0.82       139

    accuracy                           0.71       232
   macro avg       0.71      0.51      0.52       232
weighted avg       0.71      0.71      0.67       232


Service Classification:
Accuracy: 72.41%
              precision    recall  f1-score   support

    Negative       0.74      0.60      0.66        62
        None       0.68      0.75      0.71        87
    Positive       0.77      0.80      0.78        83

    accuracy                           0.72       232
   macro avg       0.73      0.71      0.72       232
weighted avg       0.73      0.72      0.72       232


Atmosphere Classification:
Accuracy: 77.59%
              precision    recall  f1-score   support

    Negative       0.00      

/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aneta/opt/miniconda3/envs/DataMining/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [15]:
# load test_data_reviews
test_data = pd.read_csv('../test_data_reviews/2_9_review.csv')    

# Predict using Logistic Regression
predictions_lr = classifier_lr.predict(test_data['text'][0:5])
#print review text and predicted ratings
for i in range(5):
    print(test_data['wiI7pd'][i])
    print("Logistic Regression Predictions:", predictions_lr[i])
    print("\n")

Cudowna obsługa , pani dokładnie sprawdzająca każde danie pod względem alergii córki , życzliwość i uśmiech Pani kelnerki . Górale i muzyka tworząca  niesamowitą swojską atmosferę .  Dania bardzo smaczne , porcje ogromne , ceny przystępne . Wrócimy na 100% Dziękujemy jeszcze raz 🥰…
Logistic Regression Predictions: {'food': 'Positive', 'service': 'Positive', 'atmosphere': 'None'}


Jedliśmy żurek barszcz z uszkami placek po zbójnicku z zestawem surówek i margaritte. Żurek znośny za to barszcz był obrzydliwy do tego uszka były rozgotowane i pokaleczone o kwaśnym smaku. Placek był bez smaku a margarita to wiór do pizzy można dostać co najwyżej ketchup. Bukiet podobno 3 surówek to mały talerzyk od espresso.
Logistic Regression Predictions: {'food': 'Negative', 'service': 'None', 'atmosphere': 'None'}


Byłem w tym miejscu pierwszy raz ok 10 lat temu i wtedy w tej restauracji było mnóstwo ludzi, pyszne jedzenie w ogromnych porcjach i widać było że knajpa żyje 😊 teraz małe porcje , drogo i n

In [16]:
# Predict using Random Forest
predictions_rf = classifier_rf.predict(test_data['text'][0:5])
#print review text and predicted ratings
for i in range(5):
    print(test_data['wiI7pd'][i])
    print("Random Forest Predictions:", predictions_rf[i])
    print("\n")

Cudowna obsługa , pani dokładnie sprawdzająca każde danie pod względem alergii córki , życzliwość i uśmiech Pani kelnerki . Górale i muzyka tworząca  niesamowitą swojską atmosferę .  Dania bardzo smaczne , porcje ogromne , ceny przystępne . Wrócimy na 100% Dziękujemy jeszcze raz 🥰…
Random Forest Predictions: {'food': 'Positive', 'service': 'Positive', 'atmosphere': 'Positive'}


Jedliśmy żurek barszcz z uszkami placek po zbójnicku z zestawem surówek i margaritte. Żurek znośny za to barszcz był obrzydliwy do tego uszka były rozgotowane i pokaleczone o kwaśnym smaku. Placek był bez smaku a margarita to wiór do pizzy można dostać co najwyżej ketchup. Bukiet podobno 3 surówek to mały talerzyk od espresso.
Random Forest Predictions: {'food': 'Negative', 'service': 'None', 'atmosphere': 'None'}


Byłem w tym miejscu pierwszy raz ok 10 lat temu i wtedy w tej restauracji było mnóstwo ludzi, pyszne jedzenie w ogromnych porcjach i widać było że knajpa żyje 😊 teraz małe porcje , drogo i niczym ni

In [17]:
# Predict using SVM
predictions_svm = classifier_svm.predict(test_data['text'][0:5])
#print review text and predicted ratings
for i in range(5):
    print(test_data['wiI7pd'][i])
    print("SVM Predictions:", predictions_svm[i])
    print("\n")

Cudowna obsługa , pani dokładnie sprawdzająca każde danie pod względem alergii córki , życzliwość i uśmiech Pani kelnerki . Górale i muzyka tworząca  niesamowitą swojską atmosferę .  Dania bardzo smaczne , porcje ogromne , ceny przystępne . Wrócimy na 100% Dziękujemy jeszcze raz 🥰…
SVM Predictions: {'food': 'Positive', 'service': 'Positive', 'atmosphere': 'None'}


Jedliśmy żurek barszcz z uszkami placek po zbójnicku z zestawem surówek i margaritte. Żurek znośny za to barszcz był obrzydliwy do tego uszka były rozgotowane i pokaleczone o kwaśnym smaku. Placek był bez smaku a margarita to wiór do pizzy można dostać co najwyżej ketchup. Bukiet podobno 3 surówek to mały talerzyk od espresso.
SVM Predictions: {'food': 'Negative', 'service': 'None', 'atmosphere': 'None'}


Byłem w tym miejscu pierwszy raz ok 10 lat temu i wtedy w tej restauracji było mnóstwo ludzi, pyszne jedzenie w ogromnych porcjach i widać było że knajpa żyje 😊 teraz małe porcje , drogo i niczym nie wyróżniające się jedze